In [0]:
%pip install rio-cogeo
%pip install rioxarray

Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 423.9/423.9 kB 30.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 111.6 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.4.0
    Not uninstalling typing-extensions at /databricks/python3/lib/python3.10/site-packages, outside environment /local_disk0/.ephemeral_nfs/envs/pythonEnv-87eba5a5-fc5e-4f5e-88d2-dd0e6c4c48df
    Can't uninstall 'typing_extensions'. No files were found to uninstall.
  Attempting uninstall: pydantic
    Found existing installation: pydantic 1.10.6
    Not uninstalling pydantic at /databricks/python3/lib/python3.10/site-packages, outside environment /local_disk0/.ephemeral_nfs/envs/pythonEnv-87eba5a5-fc5e-4f5e-88d2-dd0e6c4c48df
    Can't uninstall 'pyd

In [0]:
'''
import rio_cogeo
from rio_cogeo.cogeo import cog_translate
from rio_cogeo.profiles import cog_profiles

def noncog_to_cog_tiff(input_img, output_img):
    if rio_cogeo.cog_info(input_img).COG:
        print("The input img is already a COG!")
    else:
        print("The input img is not a COG, starting conversion of input img to COG!")
        print(cog_profiles.get("lzw"))
        # cog_translate(input_img, output_img, ** cog_profiles.get("jpeg"))
        cog_translate(input_img, output_img, **cog_profiles.get("lzw"))
        if rio_cogeo.cog_info(output_img).COG:
            print(
                f"finished converting input img to COG! The output img is saved to {output_img}"
            )


input_img = f"/Volumes/land_nib_dev/external_dev/landing_zone/33-2-426-155-31.tif"
output_img = f"/Volumes/land_nib_dev/external_dev/landing_zone/33-2-426-155-31.cog"

noncog_to_cog_tiff(input_img, output_img)
'''

'\nimport rio_cogeo\nfrom rio_cogeo.cogeo import cog_translate\nfrom rio_cogeo.profiles import cog_profiles\n\ndef noncog_to_cog_tiff(input_img, output_img):\n    if rio_cogeo.cog_info(input_img).COG:\n        print("The input img is already a COG!")\n    else:\n        print("The input img is not a COG, starting conversion of input img to COG!")\n        print(cog_profiles.get("lzw"))\n        # cog_translate(input_img, output_img, ** cog_profiles.get("jpeg"))\n        cog_translate(input_img, output_img, **cog_profiles.get("lzw"))\n        if rio_cogeo.cog_info(output_img).COG:\n            print(\n                f"finished converting input img to COG! The output img is saved to {output_img}"\n            )\n\n\ninput_img = f"/Volumes/land_nib_dev/external_dev/landing_zone/33-2-426-155-31.tif"\noutput_img = f"/Volumes/land_nib_dev/external_dev/landing_zone/33-2-426-155-31.cog"\n\nnoncog_to_cog_tiff(input_img, output_img)\n'

In [0]:
from sedona.spark import *
import pyspark.sql.functions as f
from pyspark.sql.types import ArrayType, IntegerType, LongType

# Environment
spark.sql(f"USE CATALOG land_nib_dev")
# spark.sql("DROP SCHEMA IF EXISTS raster_test CASCADE") # Sletter schema dersom det allerede eksisterer
spark.sql(f"CREATE SCHEMA IF NOT EXISTS raster_test")

# Paths
lz_path = f"/Volumes/land_nib_dev/external_dev/landing_zone"

# Table names
table_name = "raster_test.ortho_test"

In [0]:
import os
import glob

# Henter en liste med tif-filer i en mappe
file_list = glob.glob(lz_path + "/*.tif")

# Itererer gjennom fillisten og skriver ut hver fil
for file_name in file_list:
    print(f"Fil funnet: {file_name}")

# Oppretter en funksjon for å hente ut sonen
def get_sone(filnavn):
    return filnavn.split("-")[0]

# for file_name in file_list:
#    print(get_sone(file_name))

Fil funnet: /Volumes/land_nib_dev/external_dev/landing_zone/33-2-426-155-31.tif
Fil funnet: /Volumes/land_nib_dev/external_dev/landing_zone/33-2-426-155-32.tif
Fil funnet: /Volumes/land_nib_dev/external_dev/landing_zone/33-2-426-155-33.tif
Fil funnet: /Volumes/land_nib_dev/external_dev/landing_zone/33-2-427-155-01.tif
Fil funnet: /Volumes/land_nib_dev/external_dev/landing_zone/33-2-427-155-02.tif
Fil funnet: /Volumes/land_nib_dev/external_dev/landing_zone/33-2-427-155-03.tif
Fil funnet: /Volumes/land_nib_dev/external_dev/landing_zone/33-2-427-155-11.tif
Fil funnet: /Volumes/land_nib_dev/external_dev/landing_zone/33-2-427-155-12.tif
Fil funnet: /Volumes/land_nib_dev/external_dev/landing_zone/33-2-427-155-13.tif


In [0]:
import rasterio
import rasterio.merge
from rasterio.enums import Resampling
import copy
import tempfile
import shutil

bounds=None
res=None
nodata=None
precision=None

'''
def merge(input, bounds, res, nodata, precision):
    import warnings
    warnings.warn("Deprecated; Use rasterio.merge instead", DeprecationWarning)
    return rasterio.merge.merge(input, bounds, res, nodata, precision)
'''

list_of_tiffs = []
filenameCOG = 'cog.tif'
mergetiff = 'mergedRasters.tif'

# Itererer gjennom fillisten og åpner datasettene
for file_name in file_list:
    datasetNew = rasterio.open(file_name, 'r')
    out_meta = datasetNew.meta.copy()
    list_of_tiffs.append(datasetNew)

# Merger datasettene til en stor geotiff
destMerge, output_transform=rasterio.merge.merge(list_of_tiffs, bounds, res, nodata, precision)

# Stengjer alle dataset
for dataset in list_of_tiffs:
    dataset.close()

print(f"Bildedatasettet er merga til en stor geotiff {mergetiff}")

# Oppdaterer metadata på den samla geotiff-fila
out_meta.update({"driver": "GTiff",
                 "height": destMerge.shape[1],
                 "width": destMerge.shape[2],
                 "transform": output_transform})

# Oppretter temp-mappe og tempfiler
with tempfile.TemporaryDirectory() as tmp_dir:
    tmp_file = f"{tmp_dir}/{mergetiff}"
    cog_tmp_file = f"{tmp_dir}/{filenameCOG}"

    # Skriver merga fil til temp-mappa
    with rasterio.open(tmp_file, "w", **out_meta) as dest:
        dest.write(destMerge)    

    with rasterio.open(tmp_file) as src:
        #print(f"Profile {tmp_file}:\n{src.profile}")
        # Konverter til COG-formatet        
        cog_options = {
            "driver": "COG",
            "tiled": src.is_tiled,
            "blockxsize": 256,
            "blockysize": 256,
            "resampling": Resampling.nearest,
            "width": src.width,
            "height": src.height,
            "count": src.count,
            "dtype": src.read().dtype,
            "crs": src.crs,
            "transform": src.transform,
            "nodata": src.nodata
            }
        
        print(f"Konverterer {mergetiff} til file {filenameCOG} på COG-format...")

        # Skriver COG-fila til temp-mappa
        with rasterio.open(cog_tmp_file, 'w', **cog_options) as cog_dataset:         
            cog_dataset.write(src.read())
            cog_dataset.close()
        
        src.close()

    # Kopierer COG-fila til landing zone-mappen
    shutil.copy(cog_tmp_file, lz_path)

    print("Bildedatasettet er konvertert til COG-formatet.")


/home/spark-87eba5a5-fc5e-4f5e-88d2-dd/.ipykernel/50383/command-1840711720234282-2887409506:15: DeprecationWarning: Deprecated; Use rasterio.merge instead
  warnings.warn("Deprecated; Use rasterio.merge instead", DeprecationWarning)


Bildedatasettet er merga til en stor geotiff mergedRasters.tif
Konverterer mergedRasters.tif til file cog.tif på COG-format...
Bildedatasettet er konvertert til COG-formatet.


In [0]:
def get_binary_file_df(path):
    return (
        spark.read.format("binaryFile")
        .option("pathGlobFilter", "*.tif")
        .load(path)
        .withColumn("file_name", f.regexp_extract("path", "[^/]+$", 0))
        .withColumn("kartblad", f.element_at(f.split(f.col("file_name"), "\\.tif"), 1))
        .withColumn("kartblad_id", f.expr("replace(kartblad, '-', '')"))
        .withColumn("sone", f.element_at(f.split(f.col("file_name"), "-"), 1))
        .withColumn("metadata", f.expr("RS_MetaData(RS_FromGeoTiff(content))"))
    )


# Get binary file dataframe
test_dataframe = get_binary_file_df(lz_path)

# Write the new records
test_dataframe.write.format("delta").option("compression", "zstd").mode(
    "overwrite"
).saveAsTable(table_name)

# test_dataframe.show(1,truncate=False)
# Display
test_dataframe.drop("content").display()

com.databricks.backend.common.rpc.CommandSkippedException
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3(SequenceExecutionState.scala:138)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3$adapted(SequenceExecutionState.scala:133)
	at scala.collection.immutable.Range.foreach(Range.scala:158)
	at com.databricks.spark.chauffeur.SequenceExecutionState.cancel(SequenceExecutionState.scala:133)
	at com.databricks.spark.chauffeur.ExecContextState.cancelRunningSequence(ExecContextState.scala:727)
	at com.databricks.spark.chauffeur.ExecContextState.$anonfun$cancel$1(ExecContextState.scala:445)
	at scala.Option.getOrElse(Option.scala:189)
	at com.databricks.spark.chauffeur.ExecContextState.cancel(ExecContextState.scala:445)
	at com.databricks.spark.chauffeur.ChauffeurState.cancelExecution(ChauffeurState.scala:1268)
	at com.databricks.spark.chauffeur.ChauffeurState.$anonfun$process$1(ChauffeurState.scala:985)
	at com.databricks.logging.UsageL

In [0]:
test_dataframeBand1 = test_dataframe.limit(1).select(
    f.expr("RS_Band(RS_FromGeoTiff(content), Array(1))")
)
test_dataframeBand1.show(1, truncate=False)

test_dataframeBand2 = test_dataframe.limit(1).select(
    f.expr("RS_Band(RS_FromGeoTiff(content), Array(2))")
)
test_dataframeBand2.show(1, truncate=False)

test_dataframeBand3 = test_dataframe.limit(1).select(
    f.expr("RS_Band(RS_FromGeoTiff(content), Array(3))")
)
test_dataframeBand3.show(1, truncate=False)

# tiled_df = test_dataframe.limit(1).select(f.expr('RS_TileExplode(RS_FromGeoTiff(content), 1024, 1024)'))
# tiled_df.show(truncate=True)

com.databricks.backend.common.rpc.CommandSkippedException
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3(SequenceExecutionState.scala:138)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3$adapted(SequenceExecutionState.scala:133)
	at scala.collection.immutable.Range.foreach(Range.scala:158)
	at com.databricks.spark.chauffeur.SequenceExecutionState.cancel(SequenceExecutionState.scala:133)
	at com.databricks.spark.chauffeur.ExecContextState.cancelRunningSequence(ExecContextState.scala:727)
	at com.databricks.spark.chauffeur.ExecContextState.$anonfun$cancel$1(ExecContextState.scala:445)
	at scala.Option.getOrElse(Option.scala:189)
	at com.databricks.spark.chauffeur.ExecContextState.cancel(ExecContextState.scala:445)
	at com.databricks.spark.chauffeur.ChauffeurState.cancelExecution(ChauffeurState.scala:1268)
	at com.databricks.spark.chauffeur.ChauffeurState.$anonfun$process$1(ChauffeurState.scala:985)
	at com.databricks.logging.UsageL